# ECO SPARQL

Some notes on SPARQL queries used in the GeoCODES portal



## Init

Packages and functions required for this notebook

In [61]:
#@title
# !apt-get install libproj-dev proj-data proj-bin libgeos-dev
!pip install -q cython
!pip install -q cartopy
!pip install -q SPARQLWrapper
!pip install -q rdflib
!pip install -q geopandas
!pip install -q contextily==1.0rc2

In [62]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import numpy as np
import json
import geopandas
import matplotlib.pyplot as plt
import shapely

dbsparql = "http://dbpedia.org/sparql"
blaze = "http://graph.geodex.org/blazegraph/namespace/nabu/sparql"

### Helping function(s)
The following block is a SPARQL to Pandas feature.  You may need to run it to load the function per standard notebook actions.

In [63]:
#@title
def get_sparql_dataframe(service, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    sparql = SPARQLWrapper(service)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

### Set PD frame options

In [64]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

<ipython-input-64-1390d13c0cdd>:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


# SPARQL Queries



## Main Geodex Search

This is the geodex, not the geocodes search.

In [65]:
geodex = """
prefix schema: <http://schema.org/>  
prefix sschema: <https://schema.org/>  
SELECT DISTINCT ?s ?g ?url ?score  ?name ?description   
WHERE {     
  ?lit bds:search "query terms here" .    
  ?lit bds:matchAllTerms "false" .   
  ?lit bds:relevance ?score .   
  ?s ?p ?lit .       
   
  VALUES (?dataset) { ( schema:Dataset ) ( sschema:Dataset ) } 
  ?s a ?dataset .   
  ?s schema:name|sschema:name ?name .   
 
  graph ?g { 
  ?s schema:description|sschema:description ?description .  
  } 
 
  OPTIONAL { 
	?s schema:distribution|sschema:distribution ?dis .    	 
	?dis schema:contentUrl |sschema:contentUrl  ?url .    	     
  } 
 
} ORDER BY DESC(?score) 
LIMIT 10 
OFFSET 10 
"""

dfg = get_sparql_dataframe(blaze, geodex)
# df1.size
# df1.describe

In [66]:
df1.head(1)

,s,g,url,score,name,description
0,https://www.bco-dmo.org/dataset/808879,urn:gleaner:milled:bcodmo:89a9462546f406cf5aa5701e14270c245a5891a1,https://darchive.mblwhoilibrary.org/bitstream/1912/26187/2/dataset-808879_oyster-population-model-estimates__v1_README.txt,0.10825317547305485,Population model estimates for oysters (Crassostrea virginica) in the Choptank and Little Choptank Rivers,"<p>This data set contains estimates derived from a population model of oysters in the Choptank and Little Choptank Rivers. The methods and data used in the model are described in Damiano et al. (2019).</p>\r\n\r\n<p>Terms beginning with ""obs"" e.g. obs_spat, small, mk, etc. Refer to ""observed"" log-scale indices of abundance from the MD DNR Fall Dredge Survey; values were obtained from MD DNR data for fitting in the standardization model.<br />\r\nTerms beginning with ""log_pred"" e.g. log_pred_spat, small, mk, etc. Refer&nbsp;to standardized ""predicted log scale indices of relative density that have been estimated using the standardization model.<br />\r\nTerms containing ""_mil"" suffix e.g. sp_mil, sm_mil, mk_mil contain abundance estimates from the population dynamics model in millions of oysters .</p>\r\n"


## Ask query

In [67]:
ask = """
PREFIX schema:  <https://schema.org/>
	PREFIX schemaold:  <http://schema.org/>       
	ASK      
	WHERE                    {                    
	  graph   <urn:gleaner:milled:lipdverse:005a96f740da7fb3fac07936a04a86ad9d03537c> 
		  {
			{     
			  ?s schemaold:distribution|schema:distribution ?dist .    
			  ?dist  schemaold:encodingFormat|schema:encodingFormat ?type .  
			} 
			UNION {
			  VALUES (?dataset) { ( schema:Dataset ) ( schemaold:Dataset ) }
			  ?s a ?dataset .  
			  ?s  schemaold:encodingFormat|schema:encodingFormat ?type . 
			  }
		 }
		 BIND (str(?type) as ?label)                                                                                                        
		 SERVICE <http://132.249.238.169:8080/fuseki/ecrr/query> {     
		  GRAPH <http://earthcube.org/gleaner-summoned>             
		   {   
			  ?rrs schemaold:supportingData ?df.
				  ?df schemaold:encodingFormat  ?label ;
					  schemaold:position "input".	
				  ?rrs schemaold:name ?name.      
		   }                 
	   }               
	}
"""


In [68]:
# can not load ASK into DF..  just print the JSON returned
sparql = SPARQLWrapper(blaze)
sparql.setQuery(ask)
sparql.setReturnFormat(JSON)
result = sparql.query()
processed_results = json.load(result.response)
print(processed_results)

{'head': {}, 'boolean': False}


## Resource Registry to Nabu query

### WARNING:  query is old, I think this needs updating

In [69]:
rr = """
PREFIX schema:  <https://schema.org/>    
PREFIX schemaold:  <http://schema.org/>       
select DISTINCT ?rrs ?name ?curl
WHERE                    
{                    
    graph <urn:gleaner:milled:509e465d0793506b237cea8069c3cb2d276fe9c2> 
    {
    {     
        ?s schemaold:distribution|schema:distribution ?dist .    
        ?dist  schemaold:encodingFormat|schema:encodingFormat ?type .  
        ?dist schemaold:contentUrl|schema:contentUrl ?curl 
    } 
    UNION {
    VALUES (?dataset) { ( schema:Dataset ) ( schemaold:Dataset ) }
        ?s a ?dataset .  
        ?s  schemaold:encodingFormat|schema:encodingFormat ?type . 
    }
}
    BIND (str(?type) as ?label)                                                                                                        
    SERVICE <http://132.249.238.169:8080/fuseki/ecrr/query> {     
        GRAPH <http://earthcube.org/gleaner-summoned>             
        {   
            ?rrs schema:supportingData ?df.
            ?df schema:encodingFormat  ?label ;
            schema:position "input".	
            ?rrs schema:name ?name.      
        }                 
    }               
}
"""

dfrr = get_sparql_dataframe(blaze, rr)

In [70]:
dfrr.head()

,rrs,name,curl


## Resource Registry get data for tool match

In [71]:
tm = """
PREFIX schema:  <https://schema.org/>    
PREFIX schemaold:  <http://schema.org/>       
select DISTINCT ?rrs ?name ?curl
WHERE                    
{                    
    graph <urn:gleaner:milled:lipdverse:509e465d0793506b237cea8069c3cb2d276fe9c2> 
    {
    {     
        ?s schemaold:distribution|schema:distribution ?dist .    
        ?dist  schemaold:encodingFormat|schema:encodingFormat ?type .  
        ?dist schemaold:contentUrl|schema:contentUrl ?curl 
    } 
    UNION {
    VALUES (?dataset) { ( schema:Dataset ) ( schemaold:Dataset ) }
        ?s a ?dataset .  
        ?s  schemaold:encodingFormat|schema:encodingFormat ?type . 
    }
}
    BIND (str(?type) as ?label)                                                                                                        
    SERVICE <http://132.249.238.169:8080/fuseki/ecrr/query> {     
        GRAPH <http://earthcube.org/gleaner-summoned>             
        {   
            ?rrs schema:supportingData ?df.
            ?df schema:encodingFormat  ?label ;
            schema:position "input".	
            ?rrs schema:name ?name.      
        }                 
    }               
}
"""

dftm = get_sparql_dataframe(blaze, tm)

In [72]:
dftm.head()

,rrs,name,curl
0,http://n2t.net/ark:/23942/g2r331,Linked Paleo Data (LiPD) utilities software,http://lipdverse.org/Temp12k/1_0_2/Svartvatnet-Norway.Seppa.2009.lpd
1,http://n2t.net/ark:/23942/g2692r,Linked Paleo Data utilities,http://lipdverse.org/Temp12k/1_0_2/Svartvatnet-Norway.Seppa.2009.lpd
2,http://n2t.net/ark:/23942/g2733c,Linked Paleo Data (LiPD) Playground,http://lipdverse.org/Temp12k/1_0_2/Svartvatnet-Norway.Seppa.2009.lpd
3,http://n2t.net/ark:/23942/g22914,Pyleoclim,http://lipdverse.org/Temp12k/1_0_2/Svartvatnet-Norway.Seppa.2009.lpd
4,http://n2t.net/ark:/23942/g2600032,Pyleoclim,http://lipdverse.org/Temp12k/1_0_2/Svartvatnet-Norway.Seppa.2009.lpd


## Resource Registry template URL

In [73]:
tu = """
prefix sdos: <https://schema.org/>
PREFIX schemaold: <http://schema.org/>
select DISTINCT ?dataname ?appname   ?durl  ?turl ?frss
WHERE                    
{                     
   graph <urn:gleaner:milled:lipdverse:509e465d0793506b237cea8069c3cb2d276fe9c2>  {   
     ?s schemaold:distribution|sdos:distribution ?dist ;
         schemaold:name|sdos:name ?dataname  .
    ?dist  schemaold:encodingFormat|sdos:encodingFormat ?type .
          OPTIONAL {?dist sdos:contentUrl ?durl }.                    
  }                    
  BIND (str(?type) as ?label)                   
 SERVICE <http://132.249.238.169:8080/fuseki/ecrr/query> {   
   GRAPH <http://earthcube.org/gleaner-summoned>     
 
         {   
                           
           ?rrs a sdos:SoftwareApplication ;
                sdos:name ?appname ;
                sdos:supportingData ?df.
           ?df sdos:encodingFormat ?label ;
                sdos:position "input".
           ?rrs sdos:potentialAction ?act.
           ?act sdos:target ?tar.
           ?tar a sdos:EntryPoint ;
            sdos:urlTemplate ?turl.
           filter contains(?turl,"{contentURL}")
    
    }                     
 }                     
}
"""

dftu = get_sparql_dataframe(blaze, tu)

In [74]:
dftu.head()

,dataname,appname,durl,turl,frss
0,Svartvatnet-Norway.Seppa.2009,Linked Paleo Data (LiPD) Playground,http://lipdverse.org/Temp12k/1_0_2/Svartvatnet-Norway.Seppa.2009.lpd,https://lipd.net/playground?source={contentURL},None
